# ADS2 - Tutorial 11 - Aggregations

Learning Outcomes:

1.   Use Aggregation functions to explore the properties of a DataFrame
2.   Use GroupedData to perform multiple aggregations at once, over specific subsets of data




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Apache Spark uses Java, so first we must install that
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Unpack Spark from google drive
!tar xzf /content/drive/MyDrive/spark-3.3.0-bin-hadoop3.tgz

# Set up environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.3.0-bin-hadoop3"

# Install findspark, which helps python locate the psyspark module files
!pip install -q findspark
import findspark
findspark.init()

# Finally, we initialse a "SparkSession", which handles the computations
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
        
from pyspark.sql import functions as F

# Exercise 1

Upload and read the AmazonBooks.csv file from the canvas page into a DataFrame. The dataset is described [here](https://www.kaggle.com/palanjali007/amazons-top-50-bestselling-novels-20092020?select=AmazonBooks+-+Sheet1.csv).

In [3]:
### Read in the .csv data, ensure the schema is appropriate

CsvPath = '/content/AmazonBooks.csv'

# Load .csv with header, ',' seperators and inferred schema
BooksDF = spark.read\
                     .option('header', 'True')\
                     .option('sep', ',')\
                     .option('inferSchema', 'True')\
                     .csv(CsvPath)
          
# Print Schema to check
BooksDF.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Author: string (nullable = true)
 |-- User Rating: double (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Genre: string (nullable = true)



In [ ]:
### Show the top 50 books from the year 2020, ordered by User Rating
# .filter, .sort/.orderBy, .show
# Use the 'truncate=False' kwarg in .show to display the full row

# Filter by year, sort by user rating, show top 50
BooksDF.filter('Year == 2020')\
       .sort('User Rating', ascending=False)\
       .show(50, truncate=False)

+------------------------------------------------------------------------------------------------------------+------------------------+-----------+-------+-----+----+-----------+
|Name                                                                                                        |Author                  |User Rating|Reviews|Price|Year|Genre      |
+------------------------------------------------------------------------------------------------------------+------------------------+-----------+-------+-----+----+-----------+
|The Very Hungry Caterpillar                                                                                 |Eric Carle              |4.9        |35965  |7    |2020|Non Fiction|
|Brown Bear, Brown Bear, What Do You See?                                                                    | Bill Martin Jr.        |4.9        |25321  |5    |2020|Fiction    |
|Magnolia Table, Volume 2                                                                                

# Exercise 2

Aggregate [functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions) can be accessed through `pyspark.sql.functions`, this has been imported as `F` for ease of use. To perform a simple aggregation, you can call the function on a column name, then pass it to the `.select` method.

In [ ]:
### EXAMPLE: Find the highest price of any book in the dataset

# Find max price, select that column
BooksDF.select(F.max('Price')).show()

# To access the number in the DataFrame, use .first()[0]
BooksDF.select(F.max('Price')).first()[0]


+----------+
|max(Price)|
+----------+
|       105|
+----------+



105

In [ ]:
### Find the mean User Rating of all books in the dataset
### Then use the .filter method to find the mean rating for
### fiction and non fiction books
# .select, .mean, .filter

# select column mean(User Rating)
BooksDF.select(F.mean('User Rating')).show()

# filter DataFrame for Fiction and Non Fiction
BooksDF.filter('Genre == "Fiction"')\
       .select(F.mean('User Rating')).show()
BooksDF.filter('Genre == "Non Fiction"')\
       .select(F.mean('User Rating')).show()

# Alternatively:
mean_all = BooksDF.select(F.mean('User Rating')).first()[0]
mean_fiction = BooksDF.filter('Genre == "Fiction"')\
       .select(F.mean('User Rating')).first()[0]
mean_non_fiction = BooksDF.filter('Genre == "Non Fiction"')\
       .select(F.mean('User Rating')).first()[0]

print(f'Mean Rating: {mean_all:.4f}')
print(f'Mean Fiction Rating: {mean_fiction:.4f}')
print(f'Mean Non Fiction Rating: {mean_non_fiction:.4f}')


+-----------------+
| avg(User Rating)|
+-----------------+
|4.627333333333343|
+-----------------+

+-----------------+
| avg(User Rating)|
+-----------------+
|4.653992395437256|
+-----------------+

+-----------------+
| avg(User Rating)|
+-----------------+
|4.606528189910978|
+-----------------+

Mean Rating: 4.6273
Mean Fiction Rating: 4.6540
Mean Non Fiction Rating: 4.6065


In [ ]:
### Use the .count aggregate function to find the number of fiction and
### non fiction entries in the dataset

# As before, filter by genre, use count in select
# column in .count doesn't matter
BooksDF.filter('Genre == "Fiction"')\
       .select(F.count('Genre')).show()

BooksDF.filter('Genre == "Non Fiction"')\
       .select(F.count('Genre')).show()

# This is ultimately a more complex way of doing:
BooksDF.filter('Genre == "Fiction"').count()


+------------+
|count(Genre)|
+------------+
|         263|
+------------+

+------------+
|count(Genre)|
+------------+
|         337|
+------------+



263

In [ ]:
### You aren't limited to selecting a single aggregate column
### Using the .filter and .count_distinct functions, find the 
### number of unique books and authors in both genres

# Hear we filter by genre, then select the distinct of Name and Author
BooksDF.filter('Genre == "Fiction"')\
       .select(F.count_distinct('Name'),
               F.count_distinct('Author')).show()

BooksDF.filter('Genre == "Non Fiction"')\
       .select(F.count_distinct('Name'),
               F.count_distinct('Author')).show()


+--------------------+----------------------+
|count(DISTINCT Name)|count(DISTINCT Author)|
+--------------------+----------------------+
|                 177|                   100|
+--------------------+----------------------+

+--------------------+----------------------+
|count(DISTINCT Name)|count(DISTINCT Author)|
+--------------------+----------------------+
|                 214|                   179|
+--------------------+----------------------+



In [5]:
### Use the .collect_set function to get a list of all the
### unique authors in the dataset, in alphabetical order
# .select, .sort_array, .collect_set

# Call .collect_set separately so its tidier
authors_set = F.collect_set('Author')

# .sort_array sorts the results from .collect_set
BooksDF.select(F.sort_array(authors_set))\
       .show(truncate=False)

# We can see that there are some authors with leading spaces
# .ltrim can clean those up
authors_set = F.collect_set(F.ltrim('Author'))

BooksDF.select(F.sort_array(authors_set))\
       .show(truncate=False)

BooksDF.select(F.sort_array(authors_set)).first()[0]

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

['Abraham Verghese',
 'Adam Gasiewski',
 'Adam Mansbach',
 'Adam Wallace',
 'Adir Levy',
 'Admiral William H. McRaven',
 'Adult Coloring Book Designs',
 'Alan Moore',
 'Alex Michaelides',
 'Alice Schertle',
 'Allie Brosh',
 'Amelia Hepworth',
 'American Psychiatric Association',
 'American Psychological Association',
 'Amor Towles',
 'Amy Ramos',
 'Amy Shields',
 'Andy Weir',
 'Angie Grace',
 'Angie Thomas',
 'Ann Voskamp',
 'Ann Whitford Paul',
 'Anthony Bourdain',
 'Anthony Doerr',
 'Atul Gawande',
 'Audrey Niffenegger',
 'B. J. Novak',
 'Barack Obama',
 'Bessel van der Kolk M.D.',
 'Bill Martin Jr.',
 "Bill O'Reilly",
 'Bill Simmons',
 'Blue Star Coloring',
 'Bob Woodward',
 'Brandon Stanton',
 'BreneÌ\x81 Brown',
 'Brian Kilmeade',
 'Brit Bennett',
 'Bruce Springsteen',
 'Carol S. Dweck',
 'Carole P. Roman',
 'Celeste Ng',
 'Charlaine Harris',
 'Charles Duhigg',
 'Charles Krauthammer',
 'Charlie Mackesy',
 'Cheryl Strayed',
 'Chip Gaines',
 'Chip Heath',
 'Chris Cleave',
 'Chris Ky

# Exercise 3

The `.groupBy()` method produces a GroupedData object, which can in turn be used to perform aggregations. You can even group over multiple columns.

In [ ]:
### EXAMPLE: Find the mean and standard deviation of prices for each year

# group by year, feed aggregations into .agg, use .alias to rename new columns
BooksDF.groupBy('Year')\
       .agg(F.mean('Price').alias('Mean_price'),
            F.stddev('Price').alias('StdDev_price'))\
       .show()

+----+----------+------------------+
|Year|Mean_price|      StdDev_price|
+----+----------+------------------+
|2018|     10.52| 7.473053633705898|
|2015|     10.42| 7.074213016012336|
|2013|      14.6| 15.52351737966414|
|2014|     14.64| 16.56990140746351|
|2019|     10.08| 4.902477507779685|
|2020|     10.94| 5.056456773476939|
|2012|      15.3| 8.769729942552742|
|2009|      15.4|14.048022593073739|
|2016|     13.18|11.814363447662737|
|2010|     13.48| 8.164732521207117|
|2011|      15.1| 8.846191386173844|
|2017|     11.38| 8.193800543516957|
+----+----------+------------------+



In [ ]:
### Use the .groupBy method produce a single DataFrame containing
### the mean User Rating, number of entries, unique book count,
### and unique author count for Fiction and Non Fiction books
# You may find it useful to use Column expressions
# .count, .count_distinct, .mean, .groupBy, .agg

# set up the aggregations as new columns
n_entries = F.count('Genre').alias('N_entries')
n_books = F.count_distinct('Name').alias('N_books')
n_authors = F.count_distinct('Author').alias('N_authors')
mean_rating = F.mean('User Rating').alias('Mean_Rating')

# group by genre, then feed the aggregates into .agg
BooksDF.groupBy('Genre').agg(n_entries,
                             n_books,
                             n_authors,
                             mean_rating).show()

+-----------+---------+-------+---------+-----------------+
|      Genre|N_entries|N_books|N_authors|      Mean_Rating|
+-----------+---------+-------+---------+-----------------+
|    Fiction|      263|    177|      100|4.653992395437256|
|Non Fiction|      337|    214|      179|4.606528189910978|
+-----------+---------+-------+---------+-----------------+



In [ ]:
### Find the top rated book—in terms of Rating and number of Reviews—for
### each year in the dataset. Display both the name of the book, and the
### author
# .sort, .groupBy, .agg, .first
# Optional: .col, .desc

# Use col().desc() to sort the columns correctly
# .first returns the first entry in each group
BooksDF.sort(F.col('User Rating').desc(),
             F.col('Reviews').desc())\
       .groupBy('Year')\
       .agg(F.first('Name'),
            F.first('Author'))\
       .show()

+----+--------------------+-----------------+
|Year|         first(Name)|    first(Author)|
+----+--------------------+-----------------+
|2009|            The Help| Kathryn Stockett|
|2010|Unbroken: A World...|Laura Hillenbrand|
|2011|Jesus Calling: En...|      Sarah Young|
|2012|Oh, the Places Yo...|        Dr. Seuss|
|2013|Oh, the Places Yo...|        Dr. Seuss|
|2014|Oh, the Places Yo...|        Dr. Seuss|
|2015|Oh, the Places Yo...|        Dr. Seuss|
|2016|Oh, the Places Yo...|        Dr. Seuss|
|2017|Oh, the Places Yo...|        Dr. Seuss|
|2018|Oh, the Places Yo...|        Dr. Seuss|
|2019|Oh, the Places Yo...|        Dr. Seuss|
|2020|     A Promised Land|     Barack Obama|
+----+--------------------+-----------------+



In [ ]:
### As above, but do this separately for Fiction and Non Fiction
# .sort, .groupBy, .agg, .first
# Optional: .col, .desc

# As previously, but this time we get groups for each year/genre pair
BooksDF.sort(F.col('User Rating').desc(),
             F.col('Reviews').desc())\
       .groupBy('Year', 'Genre')\
       .agg(F.first('Name'),
            F.first('Author'))\
       .show(24, truncate=False)

+----+-----------+-------------------------------------------------------------------------+-----------------+
|Year|Genre      |first(Name)                                                              |first(Author)    |
+----+-----------+-------------------------------------------------------------------------+-----------------+
|2009|Fiction    |The Help                                                                 |Kathryn Stockett |
|2009|Non Fiction|Liberty and Tyranny: A Conservative Manifesto                            |Mark R. Levin    |
|2010|Fiction    |The Help                                                                 |Kathryn Stockett |
|2010|Non Fiction|Unbroken: A World War II Story of Survival, Resilience, and Redemption   |Laura Hillenbrand|
|2011|Fiction    |The Hunger Games Trilogy Boxed Set (1)                                   |Suzanne Collins  |
|2011|Non Fiction|Jesus Calling: Enjoying Peace in His Presence (with Scripture References)|Sarah Young      |
|

In [ ]:
### Group the data by author, and show their highest rated book, the number
### of times they appear in the dataset, and the number of distinct books
# .sort, .groupBy, .agg, .first, .count, .count_distinct
# Optional: .col, .desc

# Create columns and give them names
authors = F.ltrim('Author').alias('Author')
pop_book = F.first('Name').alias('Highest Rated Book')
n_books = F.count('Name').alias('Number of Entries')
n_unique_books = F.count_distinct('Name').alias('Number of Unique Books')

# Sort by popularity, group by author, feed in aggregate columns
BooksDF.sort(F.col('User Rating').desc(),
             F.col('Reviews').desc())\
       .groupBy(authors)\
       .agg(pop_book,
            n_books,
            n_unique_books)\
       .show(truncate=False)

+----------------------------------+-----------------------------------------------------------------------------------------------------------------+-----------------+----------------------+
|Author                            |Highest Rated Book                                                                                               |Number of Entries|Number of Unique Books|
+----------------------------------+-----------------------------------------------------------------------------------------------------------------+-----------------+----------------------+
|Abraham Verghese                  |Cutting for Stone                                                                                                |2                |1                     |
|Adam Gasiewski                    |Milk and Vine: Inspirational Quotes From Classic Vines                                                           |1                |1                     |
|Adam Mansbach                     |Go t